# Profitable App Profiles for the App Store and Google Play Markets

Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

## Opening and Exploring the Data

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

![img](https://s3.amazonaws.com/dq-content/350/py1m8_statista.png)

Source: [Statista](https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/)

Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

* A [data set](https://www.kaggle.com/lava18/google-play-store-apps) containing data about approximately ten thousand Android apps from Google Play is available by [this link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
* A [data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data about approximately seven thousand iOS apps from the App Store is available by [this link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

Let's start by opening the two data sets and then continue with exploring the data.

In [1]:
from csv import reader

### The App Store data set ###
opened_file = open('./AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios_data = ios[1:]

### The Google Play data set ###
opened_file = open('./googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android_data = android[1:]

PLATFORM = ({
	'name': 'Android',
	'header': android_header,
	'data': android_data,
	'data_eng': [],
	'data_final': []
	},
	{
	'name': 'iOS',
	'header': ios_header,
	'data': ios_data,
	'data_eng': [],
	'data_final': []
})

def define_platform(platform):
	for item in filter(lambda system: platform.lower() in system['name'].lower(), PLATFORM):
		return item

To make it easier to explore the two data sets, we'll first write a function named `explore_data()` that we can use repeatedly to explore rows in a more readable way. We'll also add an option for our function to show the number of rows and columns for any data set.

In [2]:
def explore_data(platform = 'android', start_row = 0, end_row = -1, rows_and_columns = True):
	dataset = define_platform(platform)

	print('{:*<100}'.format('*** Chosen platform: '))
	print(dataset['name'], '\n')

	print('{:*<100}'.format('*** Data header: '))
	print(dataset['header'], '\n')
	
	print('{:*<100}'.format('*** Data: '))
	for row in dataset['data'][start_row:end_row]:
		print(row, '\n')

	if rows_and_columns:
		print('\n*** Number of rows: ', len(dataset['data']))
		print('*** Number of columns: ', len(dataset['data'][0]))


explore_data(platform='and', start_row=0, end_row=3)

*** Chosen platform: *******************************************************************************
Android 

*** Data header: ***********************************************************************************
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

*** Data: ******************************************************************************************
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018',

We see that the Google Play data set has 10841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are `'App'`, `'Category'`, `'Reviews'`, `'Installs'`, `'Type'`, `'Price'`, and `'Genres'`.

Now let's take a look at the App Store data set.

In [3]:
explore_data(platform='ios', start_row=0, end_row=3)

*** Chosen platform: *******************************************************************************
iOS 

*** Data header: ***********************************************************************************
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

*** Data: ******************************************************************************************
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'] 


*** Number of rows:  7197
*** N

We have 7197 iOS apps in this data set, and the columns that seem interesting are: `'track_name'`, `'currency'`, `'price'`, `'rating_count_tot'`, `'rating_count_ver'`, and `'prime_genre'`. Not all column names are self-explanatory in this case, but details about each column can be found in the data set [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home).



## Deleting Wrong Data

The Google Play data set has a dedicated [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion), and we can see that [one of the discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) outlines an error for row 10472. Let's print this row and compare it against the header and another row that is correct.

In [4]:
print(android_data[10472])  # incorrect row
print('\n')
print(android_header)  # header
print('\n')
print(android_data[0])      # correct row

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


The row 10472 corresponds to the app *Life Made WI-Fi Touchscreen Photo Frame*, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5 (as mentioned in the [discussions section](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015), this problem is caused by a missing value in the `'Category'` column). As a consequence, we'll delete this row.

In [5]:
# Don't run this more than once without Restart the kernel
print(len(android_data))

del android_data[10472]

print(len(android_data))
print(android_data[10472])

10841
10840
['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


## Deleting Duplicate Entries

Now it's time to check the datasets for duplicate records for each application name:

In [6]:
duplicates = {'android': [], 'ios': []}

def find_duplicates(platform):
	platform_dict = define_platform(platform)
	data = platform_dict['data']
	
	if platform_dict['name'].lower() == 'ios':
		app_name_idx = 1
	else:
		app_name_idx = 0

	duplicates_list = []
	for app in data:
		app_name = app[app_name_idx]
		row_idx = 0
		app_duplicates = []
		for row in data:
			if app_name == row[app_name_idx]:
				app_duplicates.append({'app_name': app_name, 'row_idx': row_idx})
			row_idx += 1
		if len(app_duplicates) > 1 and app_duplicates not in duplicates_list:
			duplicates_list.append(app_duplicates)
	duplicates[platform_dict['name'].lower()] = duplicates_list
	print_duplicates(platform_dict)


def print_duplicates(platform_dict):
	duplicates_list = duplicates[platform_dict['name'].lower()]
	data = platform_dict['data']

	print(f'Found {len(duplicates_list)} duplicate apps in {platform_dict['name']} dataset (total entries - {len(data)}):\n')
	for app in duplicates_list:
		print(app)
		print(platform_dict['header'])
		idx = 0
		for item in app:
			print(data[app[idx]['row_idx']])
			idx += 1
		print('\n')


find_duplicates('and')

Found 798 duplicate apps in Android dataset (total entries - 10840):

[{'app_name': 'Coloring book moana', 'row_idx': 1}, {'app_name': 'Coloring book moana', 'row_idx': 2033}]
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']
['Coloring book moana', 'FAMILY', '3.9', '974', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


[{'app_name': 'Mcqueen Coloring pages', 'row_idx': 23}, {'app_name': 'Mcqueen Coloring pages', 'row_idx': 2111}]
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Mcqueen Coloring pages', 'ART_AND_DESIGN', 'NaN', '61', '7.0M',

In [7]:
find_duplicates('ios')

Found 2 duplicate apps in iOS dataset (total entries - 7197):

[{'app_name': 'Mannequin Challenge', 'row_idx': 2948}, {'app_name': 'Mannequin Challenge', 'row_idx': 4463}]
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['1173990889', 'Mannequin Challenge', '109705216', 'USD', '0.0', '668', '87', '3.0', '3.0', '1.4', '9+', 'Games', '37', '4', '1', '1']
['1178454060', 'Mannequin Challenge', '59572224', 'USD', '0.0', '105', '58', '4.0', '4.5', '1.0.1', '4+', 'Games', '38', '5', '1', '1']


[{'app_name': 'VR Roller Coaster', 'row_idx': 4442}, {'app_name': 'VR Roller Coaster', 'row_idx': 4831}]
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_

Let's keep the entries with the maximum number of reviews for Android apps (column index 3) and the maximum number of ratings for iOS apps (column index 5). Other duplicate entries will be deleted.

In [8]:
def delete_extra_entries(platform):
	platform_dict = define_platform(platform)
	data = platform_dict['data']
	duplicates_list = duplicates[platform_dict['name'].lower()]
	initial_entries_number = len(data)

	if platform_dict['name'].lower() == 'ios':
		key_parameter_idx = 5
	else:
		key_parameter_idx = 3

	data_deletion_rows = []
	for app in duplicates_list:
		max_key_parameter = 0
		max_key_parameter_idx = None
		app_deletion_rows = []
		for item in app:
			key_parameter = int(data[item['row_idx']][key_parameter_idx])
			if key_parameter > max_key_parameter:
				max_key_parameter = key_parameter
				if max_key_parameter_idx:
					app_deletion_rows.append(max_key_parameter_idx)
				max_key_parameter_idx = item['row_idx']
			else:
				app_deletion_rows.append(item['row_idx'])
		data_deletion_rows += app_deletion_rows

	# Preparing for extra rows deletion
	data_deletion_rows.sort()
	data_deletion_rows.reverse()

	# Deleting extra rows
	for row_idx in data_deletion_rows:
		del data[row_idx]

	print(f'Removing duplicates with a smaller key parameter is complete for {platform_dict['name']}. {
	      initial_entries_number - len(data)} entries has been deleted. Remaining entries - {len(data)}.')

# Don't run this more than once without Restart the kernel
delete_extra_entries('and')

Removing duplicates with a smaller key parameter is complete for Android. 1184 entries has been deleted. Remaining entries - 9656.


In [9]:
# Don't run this more than once without Restart the kernel
delete_extra_entries('ios')

Removing duplicates with a smaller key parameter is complete for iOS. 2 entries has been deleted. Remaining entries - 7195.


To make sure all duplicates are removed, let's repeat their search for each platform:

In [10]:
find_duplicates('and')

Found 0 duplicate apps in Android dataset (total entries - 9656):



In [11]:
find_duplicates('ios')

Found 0 duplicate apps in iOS dataset (total entries - 7195):



## Removing Non-English Apps

We're not interested in keeping the apps which are not directed toward an English-speaking audience, so we'll remove them as well.

Each character we use in a string has a corresponding number associated with it. For instance, the corresponding number for character 'a' is 97, character 'A' is 65, and character '爱' is 29,233. We can get the corresponding number of each character using the ord() built-in function.

The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

If an app name contains a character that is greater than 127, then it probably means that the app has a non-English name. To minimize the impact of data loss, we'll only remove an app if its name has more than three non-ASCII characters.

We will copy all English-language applications into separate `PLATFORM['data_eng']` constant lists, which we will work with in the future.

In [12]:
def separate_non_english(platform):
	platform_dict = define_platform(platform)
	data = platform_dict['data']
	data_eng = platform_dict['data_eng'] = []

	if platform_dict['name'].lower() == 'ios':
		app_name_idx = 1
	else:
		app_name_idx = 0

	non_english_apps = []
	row_idx = 0
	for app in data:
		non_ascii = 0

		for char in app[app_name_idx]:
			if ord(char) > 127:
				non_ascii += 1

		if non_ascii > 3:
			non_english_apps.append({'name': app[app_name_idx], 'idx': row_idx})
		else:
			PLATFORM[app_name_idx]['data_eng'].append(PLATFORM[app_name_idx]['data'][row_idx])
		
		row_idx += 1

	print(f'Separated {len(non_english_apps)} non-English apps in {platform_dict['name']} dataset (total entries before - {len(data)}, after - {len(data_eng)}):\n')

	for app in non_english_apps:
		print(app)


separate_non_english('and')

Separated 45 non-English apps in Android dataset (total entries before - 9656, after - 9611):

{'name': 'Flame - درب عقلك يوميا', 'idx': 529}
{'name': 'သိင်္ Astrology - Min Thein Kha BayDin', 'idx': 1206}
{'name': 'РИА Новости', 'idx': 2831}
{'name': 'صور حرف H', 'idx': 3165}
{'name': 'L.POINT - 엘포인트 [ 포인트, 멤버십, 적립, 사용, 모바일 카드, 쿠폰, 롯데]', 'idx': 3276}
{'name': 'RMEduS - 음성인식을 활용한 R 프로그래밍 실습 시스템', 'idx': 3491}
{'name': 'AJ렌터카 법인 카셰어링', 'idx': 4173}
{'name': 'Al Quran Free - القرآن (Islam)', 'idx': 4252}
{'name': '中国語 AQリスニング', 'idx': 4409}
{'name': '日本AV历史', 'idx': 4583}
{'name': 'Ay Yıldız Duvar Kağıtları', 'idx': 4723}
{'name': 'বাংলা টিভি প্রো BD Bangla TV', 'idx': 4882}
{'name': 'Cъновник BG', 'idx': 5038}
{'name': 'CSCS BG (в български)', 'idx': 5073}
{'name': '뽕티비 - 개인방송, 인터넷방송, BJ방송', 'idx': 5202}
{'name': 'BL 女性向け恋愛ゲーム◆俺プリクロス', 'idx': 5274}
{'name': 'SecondSecret ‐「恋を読む」BLノベルゲーム‐', 'idx': 5283}
{'name': 'BL 女性向け恋愛ゲーム◆ごくメン', 'idx': 5285}
{'name': 'あなカレ【BL】無料ゲーム', 'idx': 5286}
{'n

In [13]:
separate_non_english('ios')

Separated 1014 non-English apps in iOS dataset (total entries before - 7195, after - 6181):

{'name': '爱奇艺PPS -《欢乐颂2》电视剧热播', 'idx': 813}
{'name': '聚力视频HD-人民的名义,跨界歌王全网热播', 'idx': 1193}
{'name': '优酷视频', 'idx': 1427}
{'name': '网易新闻 - 精选好内容，算出你的兴趣', 'idx': 1518}
{'name': '淘宝 - 随时随地，想淘就淘', 'idx': 1595}
{'name': '搜狐视频HD-欢乐颂2 全网首播', 'idx': 1603}
{'name': '阴阳师-全区互通现世集结', 'idx': 1648}
{'name': '百度贴吧-全球最大兴趣交友社区', 'idx': 1782}
{'name': '百度网盘', 'idx': 1894}
{'name': '爱奇艺HD -《欢乐颂2》电视剧热播', 'idx': 1905}
{'name': '乐视视频HD-白鹿原,欢乐颂,奔跑吧全网热播', 'idx': 1916}
{'name': '万年历-值得信赖的日历黄历查询工具', 'idx': 1941}
{'name': '新浪新闻-阅读最新时事热门头条资讯视频', 'idx': 1956}
{'name': '喜马拉雅FM（听书社区）电台有声小说相声英语', 'idx': 1996}
{'name': '央视影音-海量央视内容高清直播', 'idx': 2010}
{'name': '腾讯视频HD-楚乔传,明日之子6月全网首播', 'idx': 2018}
{'name': '手机百度 - 百度一下你就得到', 'idx': 2048}
{'name': '百度视频HD-高清电视剧、电影在线观看神器', 'idx': 2049}
{'name': 'MOMO陌陌-开启视频社交,用直播分享生活', 'idx': 2095}
{'name': 'QQ 浏览器-搜新闻、选小说漫画、看视频', 'idx': 2148}
{'name': '同花顺-炒股、股票', 'idx': 2152}
{'name': '聚力视频-蓝光电

## Isolating the Free Apps

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets to the `PLATFORM['data_final']` constants.

In [14]:
# Isolating free ANDROID apps
for app in PLATFORM[0]['data_eng']:
	price = app[7]
	if price == '0':
		PLATFORM[0]['data_final'].append(app)

# Isolating free IOS apps
for app in PLATFORM[1]['data_eng']:
	price = app[4]
	if price == '0.0':
		PLATFORM[1]['data_final'].append(app)

print(f'There are {len(PLATFORM[0]['data_final'])} free Android apps saved for further analysis')
print(f'There are {len(PLATFORM[1]['data_final'])} free iOS apps saved for further analysis')

There are 8861 free Android apps saved for further analysis
There are 3220 free iOS apps saved for further analysis


## Most Common Apps by Genre

### Part One

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1) Build a minimal Android version of the app, and add it to Google Play.
2) If the app has a good response from users, we then develop it further.
3) If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the `prime_genre` column of the App Store data set, and the `Genres` and `Category` columns of the Google Play data set.

### Part Two

We'll build two functions we can use to analyze the frequency tables:

- One function to generate frequency tables that show percentages
- Another function that we can use to display the percentages in a descending order

In [15]:
def freq_table(platform, column_idx):
	platform_dict = define_platform(platform)
	data = platform_dict['data_final']

	output_table = {}
	total = 0

	for row in data:
		total += 1
		criteria = row[column_idx]
		if criteria in output_table:
			output_table[criteria] += 1
		else:
			output_table[criteria] = 1
	
	for key, value in output_table.items():
		output_table[key] = value / total * 100

	return output_table


def display_table(platform, column_idx):
	platform_dict = define_platform(platform)
	table = freq_table(platform, column_idx)
	
	result = []
	for key in table:
		key_value_as_tuple = (table[key], key)
		result.append(key_value_as_tuple)

	result_sorted = sorted(result, reverse=True)

	print(f'The frequency table for {platform_dict['name']} apps by {platform_dict['header'][column_idx].upper()} share:\n')
	for entry in result_sorted:
		print(f'{entry[1]}: {entry[0]}')

### Part Three

We start by examining the frequency table for the `prime_genre` column of the App Store data set.

In [16]:
display_table('ios', -5)

The frequency table for iOS apps by PRIME_GENRE share:

Games: 58.13664596273293
Entertainment: 7.888198757763975
Photo & Video: 4.968944099378882
Education: 3.6645962732919255
Social Networking: 3.291925465838509
Shopping: 2.608695652173913
Utilities: 2.515527950310559
Sports: 2.142857142857143
Music: 2.049689440993789
Health & Fitness: 2.018633540372671
Productivity: 1.7391304347826086
Lifestyle: 1.5838509316770186
News: 1.3354037267080745
Travel: 1.2422360248447204
Finance: 1.1180124223602486
Weather: 0.8695652173913043
Food & Drink: 0.8074534161490683
Reference: 0.5590062111801243
Business: 0.5279503105590062
Book: 0.43478260869565216
Navigation: 0.18633540372670807
Medical: 0.18633540372670807
Catalogs: 0.12422360248447205


We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the `Genres` and `Category` columns of the Google Play data set (two columns which seem to be related).

In [17]:
display_table('and', 1)

The frequency table for Android apps by CATEGORY share:

FAMILY: 18.91434375352669
GAME: 9.728021667983297
TOOLS: 8.46405597562352
BUSINESS: 4.593161042771697
LIFESTYLE: 3.9047511567543167
PRODUCTIVITY: 3.8934657487868187
FINANCE: 3.7016138133393524
MEDICAL: 3.498476469924388
SPORTS: 3.3969077982169056
PERSONALIZATION: 3.317909942444419
COMMUNICATION: 3.2389120866719328
HEALTH_AND_FITNESS: 3.080916375126961
PHOTOGRAPHY: 2.9454914795169844
NEWS_AND_MAGAZINES: 2.7987811759395105
SOCIAL: 2.663356280329534
TRAVEL_AND_LOCAL: 2.3360794492720913
SHOPPING: 2.245796185532107
BOOKS_AND_REFERENCE: 2.144227513824625
DATING: 1.8620923146371742
VIDEO_PLAYERS: 1.794379866832186
MAPS_AND_NAVIGATION: 1.3993905879697552
FOOD_AND_DRINK: 1.2413948764247829
EDUCATION: 1.1623970206522967
ENTERTAINMENT: 0.9592596772373322
LIBRARIES_AND_DEMO: 0.9366888613023362
AUTO_AND_VEHICLES: 0.9254034533348381
HOUSE_AND_HOME: 0.8238347816273558
WEATHER: 0.8012639656923597
EVENTS: 0.7109807019523756
PARENTING: 0.654553662

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the `Genres` column:

In [18]:
display_table('and', -4)

The frequency table for Android apps by GENRES share:

Tools: 8.452770567656021
Entertainment: 6.071549486513937
Education: 5.349283376594064
Business: 4.593161042771697
Productivity: 3.8934657487868187
Lifestyle: 3.8934657487868187
Finance: 3.7016138133393524
Medical: 3.498476469924388
Sports: 3.464620246021894
Personalization: 3.317909942444419
Communication: 3.2389120866719328
Action: 3.103487191061957
Health & Fitness: 3.080916375126961
Photography: 2.9454914795169844
News & Magazines: 2.7987811759395105
Social: 2.663356280329534
Travel & Local: 2.3247940413045933
Shopping: 2.245796185532107
Books & Reference: 2.144227513824625
Simulation: 2.0426588421171425
Dating: 1.8620923146371742
Arcade: 1.8508069066696762
Video Players & Editors: 1.7718090508971898
Casual: 1.760523642929692
Maps & Navigation: 1.3993905879697552
Food & Drink: 1.2413948764247829
Puzzle: 1.1285407967498025
Racing: 0.9931159011398263
Role Playing: 0.9366888613023362
Libraries & Demo: 0.9366888613023362
Auto & Veh

The difference between the `Genres` and the `Category` columns is not crystal clear, but one thing we can notice is that the `Genres` column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the `Category` column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

## Most Popular Apps by Genre on the App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the `Installs` column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the `rating_count_tot` app.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [19]:
genres_ios = freq_table('ios', -5)

for genre in genres_ios:
	total = 0
	len_genre = 0

	for app in PLATFORM[1]['data_final']:
		genre_app = app[-5]
		if genre_app == genre:
			n_ratings = float(app[5])
			total += n_ratings
			len_genre += 1
	
	avg_n_ratings = total / len_genre
	print(genre, ':', avg_n_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22812.92467948718
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [20]:
for app in PLATFORM[1]['data_final']:
	if app[-5] == 'Navigation':
		print(app[1], ':', app[5]) # Print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [21]:
for app in PLATFORM[1]['data_final']:
	if app[-5] == 'Reference':
		print(app[1], ':', app[5])  # Print name and number of ratings

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

- Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

- Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

- Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

## Most Popular Apps by Genre on Google Play

For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [22]:
display_table('and', 5) # the Installs columns

The frequency table for Android apps by INSTALLS share:

1,000,000+: 15.731858706692247
100,000+: 11.556257758717978
10,000,000+: 10.551856449610653
10,000+: 10.202008802618215
1,000+: 8.396343527818532
100+: 6.906669676108791
5,000,000+: 6.827671820336305
500,000+: 5.563706127976526
50,000+: 4.773727570251665
5,000+: 4.51416318699921
10+: 3.532332693826882
500+: 3.250197494639431
50,000,000+: 2.302223225369597
100,000,000+: 2.1329421058571265
50+: 1.9072339465071662
5+: 0.7899785577248618
1+: 0.5078433585374111
500,000,000+: 0.27084979121995256
1,000,000,000+: 0.22570815934996052
0+: 0.0451416318699921
0: 0.011285407967498025


One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to `float` — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [23]:
categories_android = freq_table('and', 1)

for category in categories_android:
	total = 0
	len_category = 0

	for app in PLATFORM[0]['data_final']:
		category_app = app[1]
		if category_app == category:
			n_installs = app[5]
			n_installs = n_installs.replace(',', '')
			n_installs = n_installs.replace('+', '')
			total += float(n_installs)
			len_category += 1

	avg_n_installs = total / len_category
	print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 121716.72258064516
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [24]:
for app in PLATFORM[0]['data_final']:
	if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+' or app[5] == '500,000,000+' or app[5] == '100,000,000+'):
		print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:

In [25]:
under_100_m = []

for app in PLATFORM[0]['data_final']:
	n_installs = app[5]
	n_installs = n_installs.replace(',', '')
	n_installs = n_installs.replace('+', '')

	if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100_000_000):
		under_100_m.append(float(n_installs))

sum(under_100_m) / len(under_100_m)

3603485.3884615386

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [26]:
for app in PLATFORM[0]['data_final']:
	if app[1] == 'BOOKS_AND_REFERENCE':
		print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [27]:
for app in PLATFORM[0]['data_final']:
	if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+' or app[5] == '500,000,000+' or app[5] == '100,000,000+'):
		print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [28]:
for app in PLATFORM[0]['data_final']:
	if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+' or app[5] == '5,000,000+' or app[5] == '10,000,000+' or app[5] == '50,000,000+'):
		print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

## Conclusions

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.